In [4]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras import layers, Input, Model, optimizers

In [12]:
from skimage import io

imag = io.imread('plots/2069-ExpBlock2_RAW.png')
x = image.img_to_array(imag)

n = preprocess_input(x)    # preprocess input for Inception v3

In [29]:
# read, preprocess, and scale images to uniform dimension
imgsize = (719, 719)    
def extract_image(image_path):
    imag = io.imread(image_path)
    x = image.img_to_array(imag)

    x = tf.keras.preprocessing.image.smart_resize(x, imgsize)
    
    n = preprocess_input(x)
    return n

In [32]:
t = extract_image('plots/2069-ExpBlock2_RAW.png')

In [36]:
t.reshape(719, 719, 1).shape

(719, 719, 1)

In [30]:
# Use InceptionV3 model for feature extraction
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(None, None, 3),
    pooling='max',
)

In [49]:
# Garcia-Ceja paper feeds CNN images with dims 200 x 100 x 4 (width x height x channels)
# Labels are one hot encoded

inputs = Input(shape=(719, 719, 1))
x = layers.Conv2D(filters=16, kernel_size=3, strides=(1, 1), activation="relu")(inputs)
x = layers.Conv2D(filters=16, kernel_size=3, strides=(1, 1), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.25)(x)
x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), activation="relu")(x)
x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(512, activation="relu")(x) 
x = layers.Dropout(0.50)(x)
out = layers.Dense(1)(x) 

model = Model(inputs=inputs, outputs=out)
model.compile(
  optimizer = 'adam',
  loss=tf.keras.losses.MeanSquaredError(),  
  metrics=['accuracy'],
)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 719, 719, 1)]     0         
                                                                 
 conv2d_314 (Conv2D)         (None, 717, 717, 16)      160       
                                                                 
 conv2d_315 (Conv2D)         (None, 715, 715, 16)      2320      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 357, 357, 16)     0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 357, 357, 16)      0         
                                                                 
 conv2d_316 (Conv2D)         (None, 355, 355, 32)      4640      
                                                           